## Dependencies
All modules and packages required for the project.

In [1]:
# Dependencies
import numpy as np
import pandas as pd
from enum import Enum
import random
import matplotlib.pyplot as plt

from util.ImageGeneration import *
from util.helper_functions import *

from collections import deque

## Task 2

In [16]:
# Softmax Regression - Task 2
class SoftmaxRegression:
    def __init__(self, X_train, y_train, X_test, y_test, X_val, y_val, lr, epsilon, regularization):
        self.n = len(X_train)                                           # of training examples
        self.d = len(X_train[0])                                        # of features
        self.X_train = X_train #np.c_[np.ones(self.n), X_train]         # Training data
        self.X_test = X_test                                            # Testing data
        self.X_val = X_val                                              # Validation data
        self.y_train = y_train                                          # Training classification Labels
        self.y_test = y_test                                            # Testing classification Labels
        self.y_val = y_val                                              # Validation classification Labels
        self.weights = np.zeros(self.d)                                 # Current parameters / weights with d rows
        self.lr = lr                                                    # Learning rate   
        self.epsilon = epsilon                                          # Early stopping difference
        self.regularization, self.Lambda, self.decay = regularization   # Type of regularization, penalty, and decay of the penalty

    # Helper methods 
    # dataset = 0 - train; 1 - val; 2 - test
    def dataset_picker(self, dataset = 0):
        if dataset == 0:
            return self.X_train, self.y_train
        elif dataset == 1:
            return self.X_test, self.y_test
        else:
            return self.X_val, self.y_val

    # Helper methods 
    def predict(self, inds=None, dataset = 0):
        """Compute h_w(x_i) for the provided weight values"""
        X, y = self.dataset_picker(dataset)
        if inds is None:
            inds = np.arange(len(X))
        
        dot_product = np.dot(X[inds], self.weights)
        return sigmoid(dot_product)

    def loss(self, Y, P):
        """Compute the current value of average loss based on predictions"""
        buffer = 1e-15
        loss = np.mean(-Y * np.log(P + buffer))
        if self.regularization == 2:
            loss += np.sum(self.Lambda * np.square(self.weights))
        return loss
    
    def accuracy(self, gold_labels, preds):
        pred_labels = self.get_pred_labels(preds)
        correct = [1 if pred == gold else 0 for pred, gold in zip(pred_labels, gold_labels)]
        count, total = sum(correct), len(correct)
        acc = round(count/total*100, 2)
        
        return acc, count, total
    
    def predict_loss_acc(self, inds=None, dataset=0):
        X, y = self.dataset_picker(dataset)
        preds = self.predict(inds, dataset)

        loss = self.loss(y, preds)
        acc, correct, total = self.accuracy(y, preds)
        
        return loss, acc
    
    def gd(self):
        """Run a single epoch of GD"""
        pass
    
    def sgd(self):
        """Run a single epoch of SGD"""
        # Shuffle data before each epoch
        indices_array = np.arange(len(self.X_train))
        random.shuffle(indices_array)
        
        for ind in indices_array:
            residual = self.predict(ind) - self.y_train[ind]
            gradient = residual * self.X_train[ind]
            if self.regularization == 2:
                gradient += 2 * self.Lambda * self.weights
            self.weights -= self.lr * gradient

    # Stochastic Gradient Descent
    def train(self, epochs, display_steps = 1, stochastic=True):
        """Run SGD until # of epochs is exceeded OR convergence"""
        prev_loss = deque([float('inf')])
        prev_acc = deque([float('inf')])
        
        self.train_losses = []
        self.val_losses = []
        self.train_accuracies = []
        self.val_accuracies = []
        print("Epoch\t\tTrainLoss\tValLoss\t\tTrainAcc\tValAcc")  
        for epoch in range(epochs):
            if stochastic:
                self.sgd()
            else: self.gd()

            loss_train, acc_train = self.predict_loss_acc(dataset=0)
            loss_val, acc_val = self.predict_loss_acc(dataset=2)
            
            self.train_losses.append(loss_train)
            self.val_losses.append(loss_val)
            self.train_accuracies.append(acc_train)
            self.val_accuracies.append(acc_val)
                        
            mean_loss = sum(prev_loss)/len(prev_loss)
            mean_acc = sum(prev_acc)/len(prev_acc)

            if epoch % display_steps == 0:
                print(f"{epoch}\t\t{round(loss_train, 3)}\t\t{round(loss_val, 3)}\t\t{acc_train}%\t\t{acc_val}%")
                #print(f"LOSS: {epoch} - train: {loss_train}; val: {loss_val}; mean: {mean_loss}")
                #print(f"ACC: {epoch} - train: {acc_train}; val: {acc_val}, mean: {mean_acc}")
            
            if abs(mean_loss - loss_val) < self.epsilon:
            #if abs(mean_acc - loss_val) < self.epsilon:
                print(f"Stopping early at epoch {epoch}")
                break
            prev_loss.append(float(loss_val))
            prev_acc.append(float(acc_val))
            if len(prev_loss) > 10:
                prev_loss.popleft()
            if len(prev_acc) > 10:
                prev_acc.popleft()

            self.Lambda *= self.decay
                
    # Model Evaluation
    def indicator(self, pred):
        """Returns label 1 if p(y == 1) > .5, 0 if p(y == 1) < .5, and breaks ties randomly"""
        if pred > .5:
            return 1
        elif pred < .5:
            return 0
        return np.random.choice([0, 1])
    
    def get_pred_labels(self, preds):
        """Converts prediction probabilities into labels"""
        for i in range(len(preds)):
            preds[i] = self.indicator(preds[i])
            
        return preds

    def test(self):
        """Compute the accuracy of the models predictions for test and training data"""
        probs_train = self.predict(dataset=0)
        acc_train, correct_train, total_train = self.accuracy(self.y_train, probs_train)
        print(f"TRAINING ACCURACY: {acc_train}%, {correct_train}/{total_train}")
        
        probs_test = self.predict(dataset=1)
        acc_test, correct_test, total_test = self.accuracy(self.y_test, probs_test)
        print(f"TESTING ACCURACY: {acc_test}%, {correct_test}/{total_test}")

        plot_data(f"Loss In Relation to Epochs ({self.n} train samples)", "Epochs", "Loss", [(self.train_losses, "Train"), (self.val_losses, "Validation")])
        plot_data(f"Accuracy In Relation to Epochs ({self.n} train samples)", "Epochs", "Accuracy", [(self.train_accuracies, "Train"), (self.val_accuracies, "Validation")])


In [17]:
# Generate and Preprocess Data
data = DataSet()
img_gen = ImageGenerator(500, dataset = data, seed = 718067190)
preprocess_data_t2(data)
image_data, third_wires = preprocess_data_t2(data)

Color.YELLOW 0
Color.YELLOW 0
Color.BLUE 0
Color.BLUE 0
Color.YELLOW 0
Color.YELLOW 0
Color.GREEN 0
Color.GREEN 0
Color.GREEN 0
Color.BLUE 0
Color.RED 0
Color.BLUE 0
Color.BLUE 0
Color.YELLOW 0
Color.YELLOW 0
Color.RED 0
Color.BLUE 0
Color.YELLOW 0
Color.BLUE 0
Color.BLUE 0
Color.YELLOW 0
Color.BLUE 0
Color.YELLOW 0
Color.GREEN 0
Color.GREEN 0
Color.YELLOW 0
Color.BLUE 0
Color.BLUE 0
Color.GREEN 0
Color.BLUE 0
Color.BLUE 0
Color.YELLOW 0
Color.GREEN 0
Color.YELLOW 0
Color.BLUE 0
Color.BLUE 0
Color.GREEN 0
Color.YELLOW 0
Color.RED 0
Color.BLUE 0
Color.YELLOW 0
Color.GREEN 0
Color.YELLOW 0
Color.YELLOW 0
Color.YELLOW 0
Color.BLUE 0
Color.RED 0
Color.GREEN 0
Color.BLUE 0
Color.YELLOW 0
Color.YELLOW 0
Color.RED 0
Color.GREEN 0
Color.RED 0
Color.YELLOW 0
Color.BLUE 0
Color.GREEN 0
Color.RED 0
Color.BLUE 0
Color.GREEN 0
Color.BLUE 0
Color.RED 0
Color.BLUE 0
Color.RED 0
Color.YELLOW 0
Color.RED 0
Color.YELLOW 0
Color.BLUE 0
Color.GREEN 0
Color.GREEN 0
Color.YELLOW 0
Color.GREEN 0
Color.GREEN 

In [14]:
print(third_wires)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 ...
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
# Set hyperparameters and train Model
lr = .01
epsilon = .0005
Lambda, decay = .01, .6
EPOCH_LIM = 500
regularization = (2, Lambda, decay)
ttv_split = train_test_validation_split(image_data, third_wires, lr, epsilon, regularization) # train, test, and validation

softmax = SoftmaxRegression(*ttv_split)
# sgd = logistic.train_stochastic(EPOCH_LIM)
# predictions = logistic.get_pred_labels(logistic.predict())

## Stashed Code
Code that does not currently have use in the notebook, but could eventually.

In [ ]:
# Stashed Code
# To reduce clutter in LogisticRegression class, I'm putting functions we currently have no need for here

if False: # so this never gets executed
    def preprocess_data(image_data, label_data):
        """Preprocess and encode image and label data for model training"""
        image_data = one_hot_encode(image_data)
        label_data = np.array(label_data)
        flattened_data = image_data.reshape(image_data.shape[0], -1)
        flattened_data = np.c_[np.ones(len(flattened_data)), flattened_data] 
        return flattened_data, label_data
    
    
        
    # Gradient Descent
    def gd(self):
        """Run Gradient Descent to find `parameters` to minimize loss"""
        # Shuffle data before each epoch
        # random.shuffle(self.examples)
        # for i in range(len(self.examples)):
        #errors = self.loss(self.labels, self.predict())
        residuals = self.predict() - self.y_train
        gradient = np.dot(self.X_train.T, residuals)
        self.weights -= self.lr * gradient
    
    
    def train_deterministic(self, epochs):
        """Run GD until # of epochs is exceeded OR convergence"""
        prev = float('inf')
        for epoch in range(epochs):
            self.gd()
            train_loss = self.loss(self.y_train, self.predict())
            if epoch % 5 == 0:
                print(f"{epoch} - Loss: {train_loss}")
                
            if prev - train_loss < self.epsilon:
                print(f"Stopping early at epoch {epoch} - Loss: {train_loss}")
                break
            prev = train_loss
            
        print(f"{epoch} - Loss: {train_loss}")